**Import libraries**

In [13]:
from google.colab import drive
drive.mount('/content/drive')

import os
import keras
import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Load the image info file and select persons who have two or more images**

In [14]:
annotations = pd.read_csv('/content/drive/MyDrive/lfw_allnames.csv')
annotations = annotations[annotations['images'] >= 2]
names = list(annotations['name'])
np.random.shuffle(names)
print('No. of persons is {}'.format(len(names)))

No. of persons is 1680


**Preparing the dataset in numpy array**

Note: This cell takes time to run

In [15]:
names_final = names.copy()
face_cascade = cv.CascadeClassifier('/content/drive/MyDrive/Face rec test drawings/haarcascade_frontalface_default.xml')  # Face detection classifiers

data = []   # whole dataset
counter = 0

for name in names:
  two_images = []    # List to hold 2 images of same person
  flag = 0

  for file in (os.listdir('/content/drive/MyDrive/lfw-deepfunneled/lfw-deepfunneled/' + str(name))[0:2]):    # Two images per person
    image = cv.imread('/content/drive/MyDrive/lfw-deepfunneled/lfw-deepfunneled/' + str(name) + '/' + str(file))
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)               # Convert to gray scale
    face = face_cascade.detectMultiScale(image, 1.3, 5)         # Detect face(s) in the image
    
    if(len(face) == 0):            # If no face is detected in the image remove that person from dataset    
      flag = 1
      names_final.remove(name)
      break

    # Crop, resize and normalize the face
    x, y, w, h = face[0]
    image = image[y:y+h, x:x+w]
    image = cv.resize(image, (64, 64), interpolation = cv.INTER_AREA)
    image = image / 255.0
    
    two_images.append(image)
  
  if (flag == 0):
    data.append(two_images)
    
    counter = counter + 1        # To see the progress of the cell
    if (counter % 154.5 == 0):
      print('\b\b\b\b\b\b{}%'.format(counter * 10 / 154.5))


data = np.array(data)
print('Total faces detected: {}'.format(2 * len(names_final)))
print(data.shape)

20.0%
40.0%
60.0%
80.0%
100.0%
Total faces detected: 3090
(1545, 2, 64, 64)


**Save the prepared data in .npy format**

In [16]:
# np.save('/content/drive/My Drive/lfw-dataset.npy', data)